In [41]:
import pandas as pd 
import process
import numpy as np 
# Jerome path : r'C:\Users\33640\OneDrive\Documents\GitHub\Portfolio_clustering_project\Data\DataBase.csv'
# Nail path : '/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/Data/DataBase.csv'
df = pd.read_csv(r'/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/Data/DataBase.csv')

df.set_index('ticker', inplace=True)

df.columns = pd.to_datetime(df.columns.str[1:], format='%Y%m%d').strftime('%d/%m/%Y')

df_cleaned = df.fillna(0) # Utilisez la méthode fillna(0) pour remplacer les NaN par 0

df_cleaned = df_cleaned.transpose() ## WE WANT COLUMNS TO BE VECTOR OF RETURN FOR A GIVEN TICKER

In [2]:
import numpy as np
4/np.exp(2)

0.5413411329464508

In [42]:
X = df_cleaned

,03/01/2000,04/01/2000,05/01/2000,06/01/2000,07/01/2000,10/01/2000,11/01/2000,12/01/2000,13/01/2000,14/01/2000,...,17/12/2020,18/12/2020,21/12/2020,22/12/2020,23/12/2020,24/12/2020,28/12/2020,29/12/2020,30/12/2020,31/12/2020
ticker,,,,,,,,,,,,,,,,,,,,,
AA,-0.013042,0.010043,0.047628,-0.011713,-0.016118,-0.032073,0.022608,-0.005249,-0.018210,-0.020109,...,-0.004498,-0.009046,0.039673,-0.025584,0.019083,-0.017156,0.006315,-0.004527,0.039099,0.002172
ABM,-0.009188,0.012346,-0.006192,0.000000,0.003091,0.000000,-0.009302,0.003130,-0.028619,0.012423,...,0.045594,-0.020819,-0.010283,-0.005448,-0.016100,-0.011338,0.009932,-0.024098,-0.002646,0.005565
ABT,-0.007117,-0.012786,0.011111,0.032553,0.028573,-0.021053,0.010850,-0.021779,-0.009217,0.000000,...,0.008772,0.001102,0.004915,0.004165,-0.013129,0.008434,-0.010796,-0.001568,-0.002303,0.012129
ADI,-0.036071,-0.044261,0.014493,-0.027719,0.033654,0.048129,-0.031921,0.034686,-0.020443,0.057086,...,0.001601,0.001246,0.006050,0.010801,-0.010972,0.002495,-0.011887,-0.010525,0.010522,0.008361
ADM,0.000000,0.005277,-0.015915,0.010695,0.005249,-0.005222,-0.015831,0.020943,0.000000,0.030459,...,-0.008269,0.007476,0.007115,-0.011021,0.006709,0.000607,-0.002010,-0.008056,0.009259,0.007367
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
XLY,-0.028632,-0.015915,-0.016645,0.004829,0.035384,-0.004103,-0.008747,-0.008282,0.000000,-0.003622,...,0.001681,-0.006541,0.003108,-0.002668,0.001970,0.001141,0.001630,-0.003188,0.002430,0.000435
XOM,-0.016647,-0.007296,0.037740,0.052842,-0.011705,-0.005210,0.002981,0.002999,0.020741,-0.020680,...,-0.011889,-0.012558,0.020227,-0.008211,0.006244,-0.001201,0.001199,-0.018486,0.006512,-0.006047
XRX,0.034169,-0.047628,0.021506,0.007926,0.023408,-0.041243,-0.015831,0.008032,0.023654,-0.026180,...,-0.006464,-0.004800,0.001789,-0.007567,0.005282,-0.014509,0.020396,-0.019023,0.010099,0.017838


In [37]:
import numpy as np

def EMA_WC(X, beta):
    """
    Calculate Exponential Moving Average Weighted Covariance matrix.

    Parameters:
        X (numpy.ndarray): Matrix of observations with shape (d, n) where n is the number of assets
                            and d is the number of days.
        beta (float): Exponential decay rate.

    Returns:
        numpy.ndarray: EMA-WC matrix.
    """
    d, _ = X.shape

    Y = (1 - beta) * beta**np.arange(d - 1, -1, -1) / (1 - beta**d)

    A = np.einsum('ti,tj->ij', Y, Y)  # Compute X * X^T
    
    W = np.diag(d * (1 - beta) * beta**(np.arange(d)[::-1]) / (1 - beta**d))  # Compute the weight matrix
    
    return np.sqrt(W) @ A  # Compute the EMA-WC matrix



In [38]:
beta = 0.99
X = df_cleaned.iloc[:1000,:].values

C = EMA_WC(X, beta)

C

ValueError: einstein sum subscripts string contains too many subscripts for operand 0

In [22]:
def EMA_CV(X, K, beta):

    X = EMA_WC(X, beta) ## compute the empirical exponential moving average of X and returns the sqrt(W) * X_tilde 


    

In [21]:
EMA

array([[5.04e-06, 7.45e-07, 9.93e-07, ..., 1.45e-03, 1.07e-03, 3.15e-04],
       [7.38e-07, 4.18e-06, 4.84e-07, ..., 1.06e-03, 6.78e-04, 2.05e-04],
       [9.73e-07, 4.79e-07, 3.32e-06, ..., 4.83e-04, 1.00e-03, 2.23e-04],
       ...,
       [1.39e-06, 1.03e-06, 4.70e-07, ..., 1.39e-02, 9.12e-04, 4.68e-04],
       [1.01e-06, 6.47e-07, 9.66e-07, ..., 9.03e-04, 4.84e-03, 2.52e-04],
       [2.94e-07, 1.93e-07, 2.12e-07, ..., 4.59e-04, 2.49e-04, 1.23e-03]])